In [1]:
import sys
import torch
import pandas as pd

/home/rastislav_papso/.conda/envs/nlp_course/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizer, RobertaModel

In [3]:
import sys
sys.path.append("..")

In [4]:
from src.model import TitleDescriptionHead

In [5]:
# BERT (https://arxiv.org/pdf/1810.04805.pdf)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# bert = BertModel.from_pretrained("bert-base-cased")

In [10]:
df = pd.read_pickle("../data/wines_seeds_embeds.pickle")

In [7]:
td_head = TitleDescriptionHead()

In [14]:
df.head(3)

,seed url,seed title,recommended url,recommended title,text only ranking (1-5),general rank,similar phrases,comments,seed description,recommended description,seed_ft,seed_fd,reco_ft,reco_fd
0,https://www.winemag.com/buying-guide/domaine-b...,Domaine Bertagna 2016 Clos de la Perrière Prem...,https://www.winemag.com/buying-guide/louis-jad...,Louis Jadot 2015 Clos de la Roche,3.0,4.0,"wine has weight, Drink from 2025","Even that the comments are not so similar, all...","This vineyard, wholly owned by the producer, l...",This grand vineyard is one of the jewels of Mo...,"[[0.091880694, -0.06518696, -0.66011435, -0.64...","[[0.15718883, 0.3467855, -0.17038564, -0.27930...","[[0.56398755, 0.94091225, -0.7593706, -0.53957...","[[0.39460817, 0.34317994, -0.32499927, -0.1747..."
1,https://www.winemag.com/buying-guide/domaine-b...,Domaine Bertagna 2016 Clos de la Perrière Prem...,https://www.winemag.com/buying-guide/albert-bi...,Albert Bichot 2016 Clos Rousseau Premier Cru (...,3.0,4.0,"ark tannins, it also has fine potential. Ripe ...",NaN,"This vineyard, wholly owned by the producer, l...",While this wine is still very young with its d...,"[[0.091880694, -0.06518696, -0.66011435, -0.64...","[[0.15718883, 0.3467855, -0.17038564, -0.27930...","[[-0.15257798, 0.32108578, -0.7898957, -0.2816...","[[0.5837172, 0.46521288, 0.038680874, 0.171878..."
2,https://www.winemag.com/buying-guide/domaine-b...,Domaine Bertagna 2016 Clos de la Perrière Prem...,https://www.winemag.com/buying-guide/franz-kel...,Franz Keller 2016 Vom Löss Pinot Noir (Baden),2.0,4.0,"plum and smoke. Black cherry, it should impro...",NaN,"This vineyard, wholly owned by the producer, l...","The nose of this wine is beautifully perfumed,...","[[0.091880694, -0.06518696, -0.66011435, -0.64...","[[0.15718883, 0.3467855, -0.17038564, -0.27930...","[[0.027803993, -0.29339287, 0.6200502, 0.58391...","[[0.22379906, 0.23264265, 0.8532843, 0.2428080..."


In [20]:
prev_seed = ""
ratings, targets = [], []

for i, seed in df.iterrows():
    if seed["seed title"] == prev_seed:
        continue

    prev_seed = seed["seed title"]
    rating, target = [], []
    seed_ft, seed_fd = torch.tensor(seed["seed_ft"]), torch.tensor(seed["seed_fd"])

    for j, reco in df.iterrows():
        reco_ft, reco_fd = torch.tensor(reco["reco_ft"]), torch.tensor(reco["reco_fd"])

        score_tt = td_head.forward(seed_ft, reco_ft)
        score_dd = td_head.forward(seed_fd, reco_fd)
        score_td = td_head.forward(seed_ft, reco_fd)
        score_dt = td_head.forward(seed_fd, reco_ft)
        total = sum([score_tt, score_dd, score_td, score_dt])

        ground_truth = seed["seed title"] == reco["seed title"]
        rating.append(total)
        target.append(ground_truth)
    
    ratings.append(rating)
    targets.append(target)

In [36]:
torch.set_printoptions(linewidth=500, edgeitems=20, precision=2)

In [23]:
r = torch.tensor(ratings)
y = torch.tensor(targets)

In [30]:
r_sort, idxs = torch.sort(r, descending=True)
y_sort = torch.gather(y, dim=1, index=idxs)

In [38]:
k = 10
top_k = y_sort[:, :k]

In [41]:
y.shape[1]

1027

In [42]:
torch.sum(top_k).item() / y.shape[1]

0.2697176241480039